In [1]:
import tensorflow as tf
tf.keras.utils.set_random_seed(42)

### Port fashion mnist dataset with (un)known intrinsic dimension:

In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images / 255.0

test_images = test_images / 255.0

In [3]:
data = train_images
data = data.reshape((len(data), -1))

### Investigate loss on different types of autoencoders (and mixes of IDs)

In [4]:
X_train = train_images
X_valid = test_images

In [49]:
ID = 200
radius = 10

input = tf.keras.layers.Input(shape=(28,28))

flatten = tf.keras.layers.Flatten(input_shape=(28, 28))(input)

latent_minus = tf.keras.layers.Dense(ID-radius, activation="relu")(flatten)
latent_id = tf.keras.layers.Dense(ID, activation="relu")(flatten)
latent_plus = tf.keras.layers.Dense(ID+radius, activation="relu")(flatten)

hidden_minus = tf.keras.layers.Dense(28 * 28)(latent_minus)
hidden_id = tf.keras.layers.Dense(28 * 28)(latent_id)
hidden_plus = tf.keras.layers.Dense(28 * 28)(latent_plus)

output_minus = tf.keras.layers.Reshape([28, 28])(hidden_minus)
output_id = tf.keras.layers.Reshape([28, 28])(hidden_id)
output_plus = tf.keras.layers.Reshape([28, 28])(hidden_plus)

model = tf.keras.Model(inputs=[input],
                       outputs=[output_minus, output_id, output_plus])

model.compile(loss="mse", optimizer="nadam",
              metrics=["RootMeanSquaredError"])
history = model.fit(X_train, X_train, epochs=10,
                         validation_data=(X_valid, X_valid))

Epoch 1/10
1875/1875 [==============================] - 9s 4ms/step - loss: 0.0445 - reshape_16_loss: 0.0150 - reshape_17_loss: 0.0148 - reshape_18_loss: 0.0147 - reshape_16_root_mean_squared_error: 0.1223 - reshape_17_root_mean_squared_error: 0.1216 - reshape_18_root_mean_squared_error: 0.1214 - val_loss: 0.0252 - val_reshape_16_loss: 0.0085 - val_reshape_17_loss: 0.0084 - val_reshape_18_loss: 0.0083 - val_reshape_16_root_mean_squared_error: 0.0924 - val_reshape_17_root_mean_squared_error: 0.0915 - val_reshape_18_root_mean_squared_error: 0.0912
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0234 - reshape_16_loss: 0.0079 - reshape_17_loss: 0.0077 - reshape_18_loss: 0.0077 - reshape_16_root_mean_squared_error: 0.0891 - reshape_17_root_mean_squared_error: 0.0880 - reshape_18_root_mean_squared_error: 0.0879 - val_loss: 0.0216 - val_reshape_16_loss: 0.0073 - val_reshape_17_loss: 0.0071 - val_reshape_18_loss: 0.0071 - val_reshape_16_root_mean_squared_error: 0

In [50]:
results = []

for i in history.history:
    results.append((i, history.history[i][-1]))

# 'ID - radius' validation loss
(ID-radius), results[8][1]

print("Result of {}: {}\n".format(ID-radius,results[8][1]))
print("Result of {}: {}\n".format(ID,results[9][1]))
print("Result of {}: {}".format(ID+radius,results[10][1]))

Result of 190: 0.006224620621651411

Result of 200: 0.0061158593744039536

Result of 210: 0.006166534963995218
